In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/A000250.csv')

def is_umbrella(df):
    body = (df['시가'] - df['종가']).abs()
    total_length = df['고가'] - df['저가']
    bull_umbrella = (df['종가'] == df['고가']) & (total_length > 3 * body)
    bear_umbrella = (df['시가'] == df['고가']) & (total_length > 3 * body)
    return (bull_umbrella | bear_umbrella).astype(int)

df["우산형"] = is_umbrella(df)
print(df)

            날짜      시가      고가      저가      종가      거래량  우산형
0     20250414  157000  157000  150700  154000   233689    0
1     20250411  152200  162900  150500  157200   282408    0
2     20250410  156000  159300  151500  156600   326907    0
3     20250409  164200  169200  145100  146400   707729    0
4     20250408  185000  189300  157900  166800  1059177    0
...        ...     ...     ...     ...     ...      ...  ...
2452  20150420   12800   13300   12600   13000   627492    0
2453  20150417   13500   13850   12900   12900  2747398    0
2454  20150416   12250   12400   12100   12300   231163    0
2455  20150415   12350   12450   12000   12150   348865    0
2456  20150414   12800   13100   11850   12200   690688    0

[2457 rows x 7 columns]


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/A000250.csv')

def is_umbrella(df):
    body = (df['시가'] - df['종가']).abs()
    total_length = df['고가'] - df['저가']
    bull_umbrella = (df['종가'] == df['고가']) & (total_length > 3 * body)
    bear_umbrella = (df['시가'] == df['고가']) & (total_length > 3 * body)
    return bull_umbrella | bear_umbrella

def is_downtrend(df):
    ma5 = df['종가'].rolling(window=5).mean()
    ma10 = df['종가'].rolling(window=10).mean()
    cond = df['종가'].shift(5)
    no_rally = df['종가'].rolling(window=5).apply(lambda x: (x <= cond.loc[x.index[0]]).all(), raw=False)

    return (ma5 < ma10) & no_rally.fillna(False)

def is_uptrend(df):
    ma5 = df['종가'].rolling(window=5).mean()
    ma10 = df['종가'].rolling(window=10).mean()

    # 5일간 종가가 5일 전 종가를 밑돌지 않음
    cond = df['종가'].shift(5)
    no_drop = df['종가'].rolling(window=5).apply(lambda x: (x >= cond.loc[x.index[0]]).all(), raw=False)

    return (ma5 > ma10) & no_drop.fillna(False)

def is_hammer_after_downtrend(df):
    downtrend = is_downtrend(df)
    uptrend = is_uptrend(df)

    body = (df['시가'] - df['종가']).abs()
    lower_shadow = df[['시가', '종가']].min(axis=1) - df['저가']

    small_body = body < (df['고가'] - df['저가']) * 0.3
    long_tail = lower_shadow > 2 * body

    return (uptrend & downtrend.shift(1).fillna(False) & small_body & long_tail)

In [ ]:
def is_hammer(df):#망치형
    downtrend = is_downtrend(df)
    body = (df['시가'] - df['종가']).abs()
    total_length = df['고가'] - df['저가']
    lower_tail = df[['시가', '종가']].min(axis=1) - df['저가']
    hammer_condition = (
        downtrend &
        (body <= total_length / 3) &
        (lower_tail >= 2 * body)
    )
    return hammer_condition.astype(int)

def is_hanging_man(df):#교수형
    uptrend = is_uptrend(df)
    body = (df['시가'] - df['종가']).abs()
    total_length = df['고가'] - df['저가']
    lower_tail = df[['시가', '종가']].min(axis=1) - df['저가']
    hanging_man_condition = (
        uptrend &
        (body <= total_length / 3) &
        (lower_tail >= 2 * body)
    )
    return hanging_man_condition.astype(int)

def is_bullish_engulfing(df):#상승장악형
    downtrend = is_downtrend(df)
    prev_open = df['시가'].shift(1)
    prev_close = df['종가'].shift(1)
    curr_open = df['시가']
    curr_close = df['종가']
    prev_bearish = prev_close < prev_open
    curr_bullish = curr_close > curr_open
    engulfing_condition = (curr_close > prev_open) & (curr_open < prev_close)
    bullish_engulfing_condition = (
        downtrend &
        prev_bearish &
        curr_bullish &
        engulfing_condition
    )
    return bullish_engulfing_condition.astype(int)

def is_bearish_engulfing(df):#하락장악형
    uptrend = is_uptrend(df)
    prev_open = df['시가'].shift(1)
    prev_close = df['종가'].shift(1)
    curr_open = df['시가']
    curr_close = df['종가']
    prev_bullish = prev_close > prev_open
    curr_bearish = curr_close < curr_open
    engulfing_condition = (curr_close < prev_open) & (curr_open > prev_close)
    bearish_engulfing_condition = (
        uptrend &
        prev_bullish &
        curr_bearish &
        engulfing_condition
    )
    return bearish_engulfing_condition.astype(int)

def is_rising_window(df):#상승창형
    prev_high = df['고가'].shift(1)
    curr_low = df['저가']
    gap_up = curr_low > prev_high
    rising_window_condition = gap_up
    return rising_window_condition.astype(int)

def is_falling_window(df):#하락창형
    prev_low = df['저가'].shift(1)
    curr_high = df['고가']
    gap_down = curr_high < prev_low
    falling_window_condition = gap_down
    return falling_window_condition.astype(int)

def is_rising_gap_tasuki(df):#상승갭 타스키형
    rising_window = is_rising_window(df)
    prev_open = df['시가'].shift(1)
    prev_close = df['종가'].shift(1)
    curr_open = df['시가']
    curr_close = df['종가']
    gap_prev_high = df['고가'].shift(2)
    prev_bullish = prev_close > prev_open
    curr_bearish = curr_close < curr_open
    prev_body = (prev_close - prev_open).abs()
    curr_body = (curr_close - curr_open).abs()
    similar_body = (curr_body >= 0.9 * prev_body) & (curr_body <= 1.1 * prev_body)
    gap_maintained = curr_close > gap_prev_high
    rising_gap_tasuki_condition = (
        rising_window.shift(1) &
        prev_bullish &
        curr_bearish &
        similar_body &
        gap_maintained
    )
    return rising_gap_tasuki_condition.astype(int)

def is_falling_gap_tasuki(df):
    falling_window = is_falling_window(df)
    prev_open = df['시가'].shift(1)
    prev_close = df['종가'].shift(1)
    curr_open = df['시가']
    curr_close = df['종가']
    gap_prev_low = df['저가'].shift(2)
    prev_bearish = prev_close < prev_open
    curr_bullish = curr_close > curr_open
    prev_body = (prev_open - prev_close).abs()
    curr_body = (curr_close - curr_open).abs()
    similar_body = (curr_body >= 0.9 * prev_body) & (curr_body <= 1.1 * prev_body)
    gap_maintained = curr_close < gap_prev_low
    falling_gap_tasuki_condition = (
        falling_window.shift(1) &
        prev_bearish &
        curr_bullish &
        similar_body &
        gap_maintained
    )
    return falling_gap_tasuki_condition.astype(int)

def is_high_gapping_play(df):#고가 갭핑플레이형
    rising_window = is_rising_window(df)
    high_gapping_conditions = []
    for lookback in range(4, 8):
        long_candle_idx = lookback + 1
        long_open = df['시가'].shift(long_candle_idx)
        long_close = df['종가'].shift(long_candle_idx)
        long_body = (long_close - long_open).abs()
        long_bullish = long_close > long_open
        small_bodies = []
        small_candles_in_range = True
        for i in range(1, lookback + 1):
            small_open = df['시가'].shift(i)
            small_close = df['종가'].shift(i)
            small_body = (small_close - small_open).abs()
            small_bodies.append(small_body)
        max_small_body = pd.concat(small_bodies, axis=1).max(axis=1)
        is_long_candle = long_body >= 3 * max_small_body
        long_body_half_high = long_open + long_body / 2
        long_body_half_low = long_open - long_body / 2
        price_constraint_met = True
        for i in range(1, lookback + 1):
            small_open = df['시가'].shift(i)
            small_close = df['종가'].shift(i)
            small_high = df[['시가', '종가']].shift(i).max(axis=1)
            small_low = df[['시가', '종가']].shift(i).min(axis=1)
            intrusion = (small_high > long_body_half_high) | (small_low < long_body_half_low)
            price_constraint_met = price_constraint_met & ~intrusion
        condition = (
            rising_window &
            long_bullish &
            is_long_candle &
            price_constraint_met
        )
        high_gapping_conditions.append(condition)
    final_condition = pd.concat(high_gapping_conditions, axis=1).any(axis=1)
    return final_condition.astype(int)

def is_low_gapping_play(df):#저가 갭플레이형
    falling_window = is_falling_window(df)
    low_gapping_conditions = []
    for lookback in range(4, 8):
        long_candle_idx = lookback + 1
        long_open = df['시가'].shift(long_candle_idx)
        long_close = df['종가'].shift(long_candle_idx)
        long_body = (long_open - long_close).abs()
        long_bearish = long_close < long_open
        small_bodies = []
        for i in range(1, lookback + 1):
            small_open = df['시가'].shift(i)
            small_close = df['종가'].shift(i)
            small_body = (small_close - small_open).abs()
            small_bodies.append(small_body)
        max_small_body = pd.concat(small_bodies, axis=1).max(axis=1)
        is_long_candle = long_body >= 3 * max_small_body
        long_body_half_high = long_open + long_body / 2
        long_body_half_low = long_open - long_body / 2
        price_constraint_met = True
        for i in range(1, lookback + 1):
            small_open = df['시가'].shift(i)
            small_close = df['종가'].shift(i)
            small_high = df[['시가', '종가']].shift(i).max(axis=1)
            small_low = df[['시가', '종가']].shift(i).min(axis=1)
            intrusion = (small_high > long_body_half_high) | (small_low < long_body_half_low)
            price_constraint_met = price_constraint_met & ~intrusion
        condition = (
            falling_window &
            long_bearish &
            is_long_candle &
            price_constraint_met
        )
        low_gapping_conditions.append(condition)
    final_condition = pd.concat(low_gapping_conditions, axis=1).any(axis=1)
    return final_condition.astype(int)

def is_rising_side_by_side(df):#상승나란히형
    rising_window_2days_ago = is_rising_window(df).shift(2)
    first_open = df['시가'].shift(1)
    first_close = df['종가'].shift(1)
    first_bullish = first_close > first_open
    first_body = first_close - first_open
    second_open = df['시가']
    second_close = df['종가']
    second_bullish = second_close > second_open
    second_body = second_close - second_open
    similar_open = (second_open >= 0.9 * first_open) & (second_open <= 1.1 * first_open)
    appropriate_body_size = (second_body >= 0.5 * first_body) & (second_body <= 2 * first_body)
    rising_side_by_side_condition = (
        rising_window_2days_ago &
        first_bullish &
        second_bullish &
        similar_open &
        appropriate_body_size
    )
    return rising_side_by_side_condition.astype(int)

def is_falling_side_by_side(df):#하락나란히형
    falling_window_2days_ago = is_falling_window(df).shift(2)
    first_open = df['시가'].shift(1)
    first_close = df['종가'].shift(1)
    first_bearish = first_close < first_open
    first_body = first_open - first_close
    second_open = df['시가']
    second_close = df['종가']
    second_bearish = second_close < second_open
    second_body = second_open - second_close
    similar_open = (second_open >= 0.9 * first_open) & (second_open <= 1.1 * first_open)
    appropriate_body_size = (second_body >= 0.5 * first_body) & (second_body <= 2 * first_body)
    falling_side_by_side_condition = (
        falling_window_2days_ago &
        first_bearish &
        second_bearish &
        similar_open &
        appropriate_body_size
    )
    return falling_side_by_side_condition.astype(int)

def is_rising_three_methods(df):#상승삼법형
    rising_three_conditions = []
    for small_candles in range(2, 5):
        first_long_idx = small_candles + 1
        first_open = df['시가'].shift(first_long_idx)
        first_close = df['종가'].shift(first_long_idx)
        first_bullish = first_close > first_open
        first_body = first_close - first_open
        second_open = df['시가']
        second_close = df['종가']
        second_bullish = second_close > second_open
        second_body = second_close - second_open
        small_bodies = []
        for i in range(1, small_candles + 1):
            small_open = df['시가'].shift(i)
            small_close = df['종가'].shift(i)
            small_body = (small_close - small_open).abs()
            small_bodies.append(small_body)
        max_small_body = pd.concat(small_bodies, axis=1).max(axis=1)
        first_is_long = first_body >= 3 * max_small_body
        second_is_long = second_body >= 3 * max_small_body
        higher_close = second_close > first_close
        higher_open = second_open > first_open
        condition = (
            first_bullish &
            second_bullish &
            first_is_long &
            second_is_long &
            higher_close &
            higher_open
        )
        rising_three_conditions.append(condition)
    final_condition = pd.concat(rising_three_conditions, axis=1).any(axis=1)
    return final_condition.astype(int)

def is_falling_three_methods(df):#하락삼법형
    falling_three_conditions = []
    for small_candles in range(2, 5):
        first_long_idx = small_candles + 1
        first_open = df['시가'].shift(first_long_idx)
        first_close = df['종가'].shift(first_long_idx)
        first_bearish = first_close < first_open
        first_body = first_open - first_close
        second_open = df['시가']
        second_close = df['종가']
        second_bearish = second_close < second_open
        second_body = second_open - second_close
        small_bodies = []
        for i in range(1, small_candles + 1):
            small_open = df['시가'].shift(i)
            small_close = df['종가'].shift(i)
            small_body = (small_close - small_open).abs()
            small_bodies.append(small_body)
        max_small_body = pd.concat(small_bodies, axis=1).max(axis=1)
        first_is_long = first_body >= 3 * max_small_body
        second_is_long = second_body >= 3 * max_small_body
        lower_close = second_close < first_close
        lower_open = second_open < first_open
        condition = (
            first_bearish &
            second_bearish &
            first_is_long &
            second_is_long &
            lower_close &
            lower_open
        )
        falling_three_conditions.append(condition)
    final_condition = pd.concat(falling_three_conditions, axis=1).any(axis=1)
    return final_condition.astype(int)

def is_rising_separating_lines(df):#상승갈림길형
    uptrend = is_uptrend(df)
    prev_open = df['시가'].shift(1)
    prev_close = df['종가'].shift(1)
    prev_bearish = prev_close < prev_open
    curr_open = df['시가']
    curr_close = df['종가']
    curr_bullish = curr_close > curr_open
    similar_open = (curr_open >= 0.9 * prev_open) & (curr_open <= 1.1 * prev_open)
    rising_separating_condition = (
        uptrend &
        prev_bearish &
        curr_bullish &
        similar_open
    )
    return rising_separating_condition.astype(int)

def is_falling_separating_lines(df):#하락갈림길형
    downtrend = is_downtrend(df)
    prev_open = df['시가'].shift(1)
    prev_close = df['종가'].shift(1)
    prev_bullish = prev_close > prev_open
    curr_open = df['시가']
    curr_close = df['종가']
    curr_bearish = curr_close < curr_open
    similar_open = (curr_open >= 0.9 * prev_open) & (curr_open <= 1.1 * prev_open)
    falling_separating_condition = (
        downtrend &
        prev_bullish &
        curr_bearish &
        similar_open
    )
    return falling_separating_condition.astype(int)

def is_doji(df):
    body = (df['시가'] - df['종가']).abs()
    total_length = df['고가'] - df['저가']
    doji_condition = body < (total_length * 0.05)
    return doji_condition.astype(int)

def is_northern_doji(df):#북향도지형
    uptrend = is_uptrend(df)
    doji = is_doji(df)-
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 5
    northern_doji_condition = (
        uptrend &
        doji &
        valid_pattern
    )
    return northern_doji_condition.astype(int)

def is_rising_long_legged_doji(df):#상승키다리도지형
    uptrend = is_uptrend(df)
    doji = is_doji(df)
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 5
    body_center = (df['시가'] + df['종가']) / 2
    high = df['고가']
    low = df['저가']
    range_third = (high - low) / 3
    middle_low = low + range_third
    middle_high = high - range_third
    in_middle_third = (body_center >= middle_low) & (body_center <= middle_high)
    rising_long_legged_doji_condition = (
        uptrend &
        doji &
        valid_pattern &
        in_middle_third
    )
    return rising_long_legged_doji_condition.astype(int)

def is_falling_long_legged_doji(df):#하락키다리도지형
    downtrend = is_downtrend(df)
    doji = is_doji(df)
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 5
    body_center = (df['시가'] + df['종가']) / 2
    high = df['고가']
    low = df['저가']
    range_third = (high - low) / 3
    middle_low = low + range_third
    middle_high = high - range_third
    in_middle_third = (body_center >= middle_low) & (body_center <= middle_high)
    falling_long_legged_doji_condition = (
        downtrend &
        doji &
        valid_pattern &
        in_middle_third
    )
    return falling_long_legged_doji_condition.astype(int)

def is_rising_gravestone_doji(df):#상승비석도지형
    uptrend = is_uptrend(df)
    doji = is_doji(df)
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 5
    body_center = (df['시가'] + df['종가']) / 2
    high = df['고가']
    low = df['저가']
    bottom_quarter_top = low + (high - low) / 4
    in_bottom_quarter = body_center <= bottom_quarter_top
    rising_gravestone_doji_condition = (
        uptrend &
        doji &
        valid_pattern &
        in_bottom_quarter
    )
    return rising_gravestone_doji_condition.astype(int)

def is_falling_gravestone_doji(df):#하락비석도지형
    downtrend = is_downtrend(df)
    doji = is_doji(df)
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 5
    body_center = (df['시가'] + df['종가']) / 2
    high = df['고가']
    low = df['저가']
    bottom_quarter_top = low + (high - low) / 4
    in_bottom_quarter = body_center <= bottom_quarter_top
    falling_gravestone_doji_condition = (
        downtrend &
        doji &
        valid_pattern &
        in_bottom_quarter
    )
    return falling_gravestone_doji_condition.astype(int)

def is_rising_dragonfly_doji(df):#상승잠자리도지형
    uptrend = is_uptrend(df)
    doji = is_doji(df)
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 5
    body_center = (df['시가'] + df['종가']) / 2
    high = df['고가']
    low = df['저가']
    top_quarter_bottom = high - (high - low) / 4
    in_top_quarter = body_center >= top_quarter_bottom
    rising_dragonfly_doji_condition = (
        uptrend &
        doji &
        valid_pattern &
        in_top_quarter
    )
    return rising_dragonfly_doji_condition.astype(int)

def is_falling_dragonfly_doji(df):#하락잠자리도지형
    downtrend = is_downtrend(df)
    doji = is_doji(df)
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 5
    body_center = (df['시가'] + df['종가']) / 2
    high = df['고가']
    low = df['저가']
    top_quarter_bottom = high - (high - low) / 4
    in_top_quarter = body_center >= top_quarter_bottom
    falling_dragonfly_doji_condition = (
        downtrend &
        doji &
        valid_pattern &
        in_top_quarter
    )
    return falling_dragonfly_doji_condition.astype(int)

def is_rising_three_stars(df):#상승삼별형
    doji_today = is_doji(df)
    doji_1day_ago = is_doji(df).shift(1)
    doji_2day_ago = is_doji(df).shift(2)
    three_consecutive_doji = doji_today & doji_1day_ago & doji_2day_ago
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 6
    high_20days = df['고가'].rolling(window=20, min_periods=1).max()
    high_today = df['고가']
    high_1day_ago = df['고가'].shift(1)
    high_2day_ago = df['고가'].shift(2)
    high_breakout = (
        (high_today > high_20days) |
        (high_1day_ago > high_20days.shift(1)) |
        (high_2day_ago > high_20days.shift(2))
    )
    rising_three_stars_condition = (
        three_consecutive_doji &
        valid_pattern &
        high_breakout
    )
    return rising_three_stars_condition.astype(int)

def is_falling_three_stars(df):#하락삼별형
    doji_today = is_doji(df)
    doji_1day_ago = is_doji(df).shift(1)
    doji_2day_ago = is_doji(df).shift(2)
    three_consecutive_doji = doji_today & doji_1day_ago & doji_2day_ago
    doji_count_20days = is_doji(df).rolling(window=20, min_periods=1).sum()
    valid_pattern = doji_count_20days < 6
    low_20days = df['저가'].rolling(window=20, min_periods=1).min()
    low_today = df['저가']
    low_1day_ago = df['저가'].shift(1)
    low_2day_ago = df['저가'].shift(2)
    low_breakdown = (
        (low_today < low_20days) |
        (low_1day_ago < low_20days.shift(1)) |
        (low_2day_ago < low_20days.shift(2))
    )
    falling_three_stars_condition = (
        three_consecutive_doji &
        valid_pattern &
        low_breakdown
    )
    return falling_three_stars_condition.astype(int)
